In [ ]:
## Дерево решений

In [ ]:
Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [1]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print 'invalid criterion name'
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print 'invalid max_features name'
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return # Ваш код
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return # Ваш код 

    def __misclass(self, l_c, l_s, r_c, r_s):
        return # Ваш код

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return # Ваш код
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return # Ваш код

    def __get_feature_ids_N(self, n_feature):
        return # Ваш код
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # Сортирует X и Y, т.к. при выборе способа разбиения, необходиммо отсортировать данные по атрибуту.
        # На 2 строчке, получается список всех классов по которым будет происходить разбиение.
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        # Что делает этот блок кода?
        # Находит границу, разделения классов для этого:
        # Отрезает на отсортированном массиве от начала и конца min_samples_split
        # И на обрезанном массиве ищет границу, где один класс переходит в другой
        # Затем прибавляет min_samples_split + 1 т.к. мы щем индекс на отсортированном массиве а не на отрезанном
        # Если на всём отрезке расположены элементы одного класса, то r_border_ids - пустой массив
        splitted_sorted_y = sorted_y[self.min_samples_split:-self.min_samples_split]
        r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (self.min_samples_split + 1)
        
        if len(r_border_ids) == 0:
            return float('+inf'), None
        
        # Что делает этот блок кода?
        # Считает количество элементов левого класса
        eq_el_count = r_border_ids - np.append([self.min_samples_split], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] + np.bincount(sorted_y[:self.min_samples_split], minlength=class_number)
        
        # Что делает этот блок кода?
        # Считается количество элементов левого и правого классов l_class_count, r_class_count в левом и правом и отрезке соответственно
        # Затем считается количество всего элементов в левом и правом отрезке l_sizes, r_sizes
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # запускает функцию, которая используется для определения границы, которая возвращает массив значений, для разных точек
        # Затем возвращает индекс наименьшего элемента
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        idx = np.argmin(gs)
    
        # Что делает этот блок кода?
        # Возвращает индекс, по которому произошло разделение массива элементов и значение, которое является разделителем.
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        pass
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('invalid criterion name')? (<ipython-input-2-1f06c86868fe>, line 18)

In [3]:
df = pd.read_csv('./cs-training.csv', sep=',').dropna()
df.shape

(120269, 11)

In [78]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])



min_samples_split = 1
sample = y[8:23] #np.array([1,1,1,1,2,2,2,2,3,3,3,3]) #y[8:23]
sorted_y = sample[sample.argsort()]
splitted_sorted_y = sorted_y[min_samples_split:-min_samples_split]
r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (min_samples_split + 1)

print(sample)
print(sorted_y)
print(splitted_sorted_y)
print(np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:]) )
print(r_border_ids)

class_number = 3

print('---------')
eq_el_count = r_border_ids - np.append([min_samples_split], r_border_ids[:-1])
print( np.append([min_samples_split], r_border_ids[:-1]))
one_hot_code = np.zeros((r_border_ids.shape[0], class_number))

print(eq_el_count)
print(one_hot_code)
one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
print(one_hot_code)
class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
print(class_increments)
class_increments[0] = class_increments[0] + np.bincount(sorted_y[:min_samples_split], minlength=class_number)
print(class_increments)


print('------------')
l_class_count = np.cumsum(class_increments, axis=0)
print(l_class_count)
print(np.bincount(sample))

# r_class_count = np.bincount(sorted_y) - l_class_count
l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
#r_sizes = sorted_y.shape[0] - l_sizes

print(l_class_count)
#print(r_class_count)
print(l_sizes)
print(r_sizes)
# x = x[:5]
# y = y[:5]
# 
# sorted_idx = x.argsort()
# print(x)
# print(sorted_idx)
# print(x[sorted_idx])
# print(y[sorted_idx])

[0 0 0 1 0 0 0 0 0 0 1 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 1 1]
(array([10]),)
[12]
---------
[1]
[11]
[[0. 0. 0.]]
[[1. 0. 0.]]
[[11.  0.  0.]]
[[12.  0.  0.]]
------------
[[12.  0.  0.]]
[12  3]
[[12.  0.  0.]]
[[12]]


/home/carefreeslacker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/home/carefreeslacker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


NameError: name 'r_sizes' is not defined

In [ ]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [ ]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

## Проверка качества работы

In [ ]:
gkf = KFold(n_splits=5, shuffle=True)

In [ ]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

In [ ]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

# Применить для задачи Titanic 